```mermaid
flowchart LR
    A[dataset_builder] -->|原始样本| B[teacher_labeler]
    B -->|生成标签| C[train_lora]
    C -->|LoRA 模型| D[evaluate]
```

以上示意图展示了各模块的协作关系：
- `dataset_builder`：收集与清洗数据，输出原始训练样本；
- `teacher_labeler`：调用教师模型，为样本添加 `prediction`、`analysis` 等标签；
- `train_lora`：基于带标签数据执行 LoRA 微调；
- `evaluate`：加载微调模型，在验证集上评估效果。

### LoRA 原理概述
LoRA（Low-Rank Adaptation）将权重增量表示为低秩分解 $\Delta W = B A$，仅训练小矩阵 $A,B$，而预训练权重 $W_0$ 保持冻结不更新，从而在保持原模型能力的同时大幅减少可训练参数和显存开销。

在本示例中，可选参数 `rope_factor` 会按比例缩放旋转位置编码（RoPE）的基础频率，以此扩展模型可接受的上下文长度；例如 `rope_factor=2` 时，理论上可将上下文窗口扩大到原来的两倍。

### 金融场景下的动机示例
在金融任务中，模型需理解长篇财报、行情序列并生成投资建议。通过 LoRA 微调，我们只需在少量行业数据上训练小规模适配器，即可快速获得专用模型；配合 `rope_factor` 扩展后的 RoPE，模型还能一次性读入更长的时间区间或多份文档。这为后续的数据构建、教师标注、训练与评估提供了现实动机。

# 大模型 LoRA 微调流程演示

本示例 Notebook 展示金融股票领域大模型微调的完整流程，包括数据集构建、教师模型标注、LoRA 微调训练以及模型评估。示例使用 `test4` 模块中的代码，并利用模拟数据和模型以便在轻量环境下演示整体流程。

## 背景介绍
- **LoRA** 通过在预训练模型的部分权重上插入低秩矩阵，在冻结原模型参数的情况下完成高效微调。
- **RoPE 长上下文扩展**: 通过设置 `rope_factor` 扩展旋转位置编码，使模型支持更长的输入序列。
- **Token 截断**: 在构造数据集和训练时确保输入长度不超过最大上下文窗口。

## 步骤1：构建模拟数据集
下面构造两只股票的30日K线数据，并使用 `dataset_builder.build_dataset` 生成训练/验证样本。为避免真实网络请求，我们 monkey-patch `_fetch_kline` 函数返回模拟数据。

In [ ]:
import os, sys, json
sys.path.append(os.getcwd())  # 使 test4 模块可导入

from test4 import dataset_builder

import pandas as pd
import numpy as np

dates = pd.date_range('2023-01-01', periods=30, freq='D').strftime('%Y-%m-%d')
prices1 = 100 * np.cumprod(1 + np.random.normal(0, 0.01, size=30))
prices2 = 50 * np.cumprod(1 + np.random.normal(0, 0.01, size=30))

df_stock1 = pd.DataFrame({
    'date': dates,
    'open': prices1,
    'close': prices1 + np.random.normal(0, 0.5, size=30),
    'high': prices1 + np.random.normal(0, 1, size=30),
    'low':  prices1 - np.random.normal(0, 1, size=30),
    'volume': np.random.uniform(1000, 10000, size=30)
})

df_stock2 = pd.DataFrame({
    'date': dates,
    'open': prices2,
    'close': prices2 + np.random.normal(0, 0.5, size=30),
    'high': prices2 + np.random.normal(0, 1, size=30),
    'low':  prices2 - np.random.normal(0, 1, size=30),
    'volume': np.random.uniform(1000, 10000, size=30)
})

def dummy_fetch_kline(code: str, days: int):
    if code == '000001':
        return df_stock1.tail(days).reset_index(drop=True)
    elif code == '000002':
        return df_stock2.tail(days).reset_index(drop=True)
    else:
        return None

dataset_builder._fetch_kline = dummy_fetch_kline

train_samples, val_samples = dataset_builder.build_dataset(['000001','000002'], days=30, window=30, val_ratio=0.2, seed=42)
print(f"训练样本数: {len(train_samples)}, 验证样本数: {len(val_samples)}")
if train_samples:
    example_prompt = dataset_builder.format_prompt(train_samples[0])
    print(example_prompt[:120] + '...')

## 步骤2：教师模型标注
使用 `teacher_labeler` 将 Prompt 列表转换为带有 `prediction`、`analysis` 和 `advice` 字段的答案。这里的 `call_teacher` 函数被替换为返回固定内容的模拟教师模型。

In [ ]:
from test4 import teacher_labeler

def dummy_call_teacher(prompt: str):
    if '000001' in prompt:
        pred = '上涨'
    else:
        pred = '下跌'
    return json.dumps({'prediction': pred, 'analysis': '模拟分析', 'advice': '模拟建议'}, ensure_ascii=False)

teacher_labeler.call_teacher = dummy_call_teacher

train_prompts = [dataset_builder.format_prompt(s) for s in train_samples]
val_prompts = [dataset_builder.format_prompt(s) for s in val_samples]

train_records = teacher_labeler.label_samples(train_prompts, output_file='labeled_data.jsonl')
val_records = teacher_labeler.label_samples(val_prompts, output_file='val_labeled_data.jsonl')

print(f"已标注训练样本数: {len(train_records)}, 验证样本数: {len(val_records)}")
print(json.dumps(train_records[0], ensure_ascii=False, indent=2))

## 步骤3：LoRA 微调训练
下面调用 `train_lora.main` 进行一次简化训练。为了便于快速运行，使用体积很小的 `sshleifer/tiny-gpt2` 作为基础模型，并将训练步数限制为 1。

In [ ]:
from test4 import train_lora

if not hasattr(train_lora, 'orig_BitsAndBytesConfig'):
    train_lora.orig_BitsAndBytesConfig = train_lora.BitsAndBytesConfig

def safe_bnb_config(**kwargs):
    try:
        import torch
        if torch.cuda.is_available():
            return train_lora.orig_BitsAndBytesConfig(**kwargs)
    except ImportError:
        pass
    return None

train_lora.BitsAndBytesConfig = safe_bnb_config

cfg = train_lora.TrainConfig(
    base_model='sshleifer/tiny-gpt2',
    data_path='labeled_data.jsonl',
    eval_path='val_labeled_data.jsonl',
    output_dir='lora_demo',
    epochs=1,
    max_steps=1,
    batch_size=1,
    max_len=4096,
    rope_factor=2.0
)

train_lora.main(cfg)
print('LoRA 微调完成')

## 步骤4：模型评估
示例中我们基于验证集计算 BLEU 分数。为了展示流程，假设基座模型输出为空 JSON，而微调模型能够准确复现教师答案。

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

prompts = [rec['prompt'] for rec in val_records]
refs = [json.dumps(rec['label'], ensure_ascii=False, sort_keys=True) for rec in val_records]

base_preds = ['{}' for _ in prompts]
# 模拟微调模型输出等同于参考答案
with open('val_labeled_data.jsonl', encoding='utf-8') as f:
    tuned_preds = [json.loads(line)['label'] for line in f]
    tuned_preds = [json.dumps(t, ensure_ascii=False, sort_keys=True) for t in tuned_preds]

smooth = SmoothingFunction().method4
base_bleu = sum(sentence_bleu([r.split()], p.split(), smoothing_function=smooth) for r,p in zip(refs, base_preds)) / len(refs)
tuned_bleu = sum(sentence_bleu([r.split()], p.split(), smoothing_function=smooth) for r,p in zip(refs, tuned_preds)) / len(refs)
print(f"BLEU 未微调: {base_bleu:.4f}, 微调后: {tuned_bleu:.4f}")

## 总结
通过以上步骤，我们演示了基于 `test4` 模块的完整 LoRA 微调流程：从构建股票K线数据集、教师模型标注，到对学生模型进行 LoRA 微调并进行指标评估。该流程能够在有限资源下让模型学习金融领域知识。